# Multi-Agent Research Team with CrewAI + Qdrant

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thierrypdamiba/qdrant-etl-cookbook/blob/main/notebooks/agents/crewai_qdrant.ipynb)

Build a multi-agent team where agents share a Qdrant knowledge base. One agent retrieves context, another synthesizes answers.

**Requirements:** Set `OPENAI_API_KEY` environment variable.

In [ ]:
!pip install -q "qdrant-client[fastembed]>=1.13,<1.16" crewai crewai-tools

In [ ]:
import os
from qdrant_client import QdrantClient

qdrant = QdrantClient(":memory:")

# Seed knowledge base
documents = [
    "Qdrant supports HNSW indexing with configurable m and ef_construct parameters.",
    "Scalar quantization in Qdrant reduces memory usage by 4x with int8 compression.",
    "Binary quantization provides 32x memory reduction, best for 1536-dim vectors.",
    "Multi-tenancy is achieved through payload-based filtering with tenant_id fields.",
    "Qdrant Cloud offers managed clusters with automatic scaling and backups.",
    "Hybrid search combines dense vectors with sparse BM25 vectors for better recall.",
    "Payload indexes on keyword, integer, and geo fields speed up filtered search.",
    "Snapshots can be created per-collection or as full storage snapshots.",
]

qdrant.add(
    collection_name="qdrant_knowledge",
    documents=documents,
)
print(f"Loaded {len(documents)} documents into knowledge base")

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import tool

@tool("Search Qdrant Knowledge Base")
def search_knowledge(query: str) -> str:
    """Search the Qdrant knowledge base for relevant information."""
    results = qdrant.query(
        collection_name="qdrant_knowledge",
        query_text=query,
        limit=3,
    )
    return "\n".join([r.document for r in results])

In [ ]:
# Define agents
researcher = Agent(
    role="Research Analyst",
    goal="Find relevant technical information from the knowledge base",
    backstory="You are an expert at searching and retrieving relevant technical documentation.",
    tools=[search_knowledge],
    verbose=True,
)

writer = Agent(
    role="Technical Writer",
    goal="Synthesize research into clear, actionable recommendations",
    backstory="You turn technical findings into practical advice for developers.",
    verbose=True,
)

In [ ]:
# Define tasks
research_task = Task(
    description="Search the knowledge base for information about optimizing Qdrant for a high-traffic production deployment with millions of vectors.",
    expected_output="A list of relevant findings about performance optimization, quantization, and scaling.",
    agent=researcher,
)

writing_task = Task(
    description="Based on the research findings, write a short deployment checklist for optimizing Qdrant in production.",
    expected_output="A numbered checklist of 5-7 actionable steps.",
    agent=writer,
)

# Run the crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    verbose=True,
)

result = crew.kickoff()
print("\n=== Final Output ===")
print(result)